In [1]:
import tensorflow as tf
import datetime

In [2]:
tf.__version__

'2.0.0'

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [4]:
train_images.shape

(60000, 28, 28)

In [5]:
train_images = tf.expand_dims(train_images, -1)
test_images = tf.expand_dims(test_images, -1)
train_images.shape

TensorShape([60000, 28, 28, 1])

In [6]:
test_images.shape

TensorShape([10000, 28, 28, 1])

In [7]:
train_images = tf.cast(train_images/255, tf.float32)
test_images = tf.cast(test_images/255, tf.float32)

train_labels = tf.cast(train_labels, tf.int64)
test_labels = tf.cast(test_labels, tf.int64)

In [8]:
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

In [9]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [10]:
dataset = dataset.repeat().shuffle(60000).batch(128)

In [11]:
test_dataset = test_dataset.repeat().batch(128)

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3, 3], activation='relu', input_shape = (None, None, 1)),
    tf.keras.layers.Conv2D(32, [3, 3], activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10, activation='softmax')   
])

In [13]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [14]:
import os
log_dir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%dataset-%H%M%S"))

In [15]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq = 1)     #调用Tensorboard记录日志

In [16]:
def lr_sche(epoch):                      #定义学习速率函数
    learning_rate = 0.2
    if epoch > 5:
        learning_rate = 0.02
    if epoch >10:
        learning_rate = 0.01
    if epoch >20:
        learning_rate = 0.005
    tf.summary.scalar('learning_rate', data = learning_rate, step = epoch)    #调用summary.scalar方法将自适应learning_rate写入log
    return learning_rate    

In [17]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_sche)  #调用学习速率函数作为参数，来控制学习速率

把学习速率的变化写到日志中

In [18]:
file_writer = tf.summary.create_file_writer(log_dir + '/lr')

In [19]:
file_writer.set_as_default()

In [20]:
model.fit(dataset,
          epochs=5,
          steps_per_epoch=60000//128,
          validation_data=test_dataset,
          validation_steps=10000//128,
          callbacks=[tensorboard_callback, lr_callback]    #使用tensorboard回调函数
)

Train for 468 steps, validate for 78 steps
Epoch 1/5
468/468 [==============================] - 10s 22ms/step - loss: 2.3488 - acc: 0.1033 - val_loss: 2.3089 - val_acc: 0.0982
Epoch 2/5
468/468 [==============================] - 7s 16ms/step - loss: 2.3116 - acc: 0.1044 - val_loss: 2.3149 - val_acc: 0.0982
Epoch 3/5
468/468 [==============================] - 7s 16ms/step - loss: 2.3128 - acc: 0.1014 - val_loss: 2.3300 - val_acc: 0.0975
Epoch 4/5
468/468 [==============================] - 7s 15ms/step - loss: 2.3126 - acc: 0.1021 - val_loss: 2.3072 - val_acc: 0.1010
Epoch 5/5
468/468 [==============================] - 8s 17ms/step - loss: 2.3134 - acc: 0.1023 - val_loss: 2.3123 - val_acc: 0.1032


Tensorboard日志文献的显示

In [21]:
%load_ext tensorboard
%matplotlib inline

In [22]:
#%tensorboard --logdir logs

# 在自定义训练中使用tensorboard

In [23]:
optimizer = tf.keras.optimizers.Adam() 

In [24]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [25]:
def loss(model, x, y):
    y_ = model(x)
    return loss_func(y, y_)   

In [26]:
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')

test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [27]:
def train_step(model, images, labels):                               #一步优化过程
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels, pred)
        
        #loss_step = loss(model, images, labels)                      #计算损失函数        
    grads = t.gradient(loss_step, model.trainable_variables)         #计算损失函数与可训练参数之间的梯度
    optimizer.apply_gradients(zip(grads, model.trainable_variables)) #优化函数应用梯度对可训练参数进行优化
    train_loss(loss_step)
    train_accuracy(labels, pred)

In [28]:
def test_step(model, images, labels):                          
    pred = model(images)
    loss_step = loss_func(labels, pred)
    test_loss(loss_step)
    test_accuracy(labels, pred)

In [29]:
current_time = datetime.datetime.now().strftime("%Y%m%dataset-%H%M%S")

In [31]:
#创建train和test的文件编写器
train_log_dir = 'log/gradient_tape' + current_time + '/train'
test_log_dir = 'log/gradient_tape' + current_time + '/test'

train_writer = tf.summary.create_file_writer(train_log_dir)
test_writer = tf.summary.create_file_writer(test_log_dir)

In [32]:
def train():
    for epoch in range(10):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(model, images, labels)
        print('epoch{} loss is {}, train_accuracy is {}'.format(epoch, train_loss.result(), train_accuracy.result()))
        with train_writer.as_default():       #train_writer作为默认的文件编写器
            tf.summary.scalar('loss', train_loss.result(), step=epoch)               #scalar方法收集标量值，使用file_writer写到磁盘上
            tf.summary.scalar('acc', train_accuracy.result(), step=epoch)
        
        for (batch, (images, labels)) in enumerate(test_dataset):
            test_step(model, images, labels)
        print('epoch{} loss is {}, test_accuracy is {}'.format(epoch, test_loss.result(), test_accuracy.result()))
        with test_writer.as_default():       #train_writer作为默认的文件编写器
            tf.summary.scalar('loss', test_loss.result(), step=epoch)               #scalar方法收集标量值，使用file_writer写到磁盘上
            tf.summary.scalar('acc', test_accuracy.result(), step=epoch)
                
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

In [34]:
train()

KeyboardInterrupt: 